In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite
import community
import itertools

team_data = pd.read_csv('LeagueofLegends.csv')

teams_blue = team_data[['League','Year','Season','Type','blueTeamTag','blueTop','blueJungle','blueMiddle', 'blueADC', 'blueSupport']]
teams_red = team_data[['League','Year','Season','Type','redTeamTag','redTop','redJungle','redMiddle', 'redADC', 'redSupport']]

teams_blue.columns = ['League','Year','Season','Type','TeamTag','Top','Jungle','Middle', 'ADC', 'Support']
teams_red.columns = ['League','Year','Season','Type','TeamTag','Top','Jungle','Middle', 'ADC', 'Support']

teams = pd.concat([teams_blue,teams_red])

teams = teams.dropna()
teams.shape

(15165, 10)

In [2]:
# Some cleaning

teams.loc[(teams['TeamTag']=='ROX') & (teams['League']=='LCK'),'TeamTag'] = 'ROXTigers'
teams['Top'] = teams['Top'].str.lower()
teams['Jungle'] = teams['Jungle'].str.lower()
teams['Middle'] = teams['Middle'].str.lower()
teams['ADC'] = teams['ADC'].str.lower()
teams['Support'] = teams['Support'].str.lower()

teams_fix = teams.copy()
teams_fix.loc[teams_fix['Top']=='wunderwear', 'Top'] = 'wunder'
teams_fix.loc[teams_fix['Top']=='lil4c', 'Top'] = 'lilac'
teams_fix.loc[teams_fix['Top']=='s achie', 'Top'] = 'achie'
teams_fix.loc[teams_fix['Top']=='m1st', 'Top'] = 'mist'

teams_fix.loc[teams_fix['Jungle']=='crumbzz', 'Jungle'] = 'crumbz'
teams_fix.loc[teams_fix['Jungle']=='amazingx', 'Jungle'] = 'amazing'
teams_fix.loc[teams_fix['Jungle']=='diamond', 'Jungle'] = 'diamondprox'
teams_fix.loc[teams_fix['Jungle']=='flaw', 'Jungle'] = 'flawless'
teams_fix.loc[teams_fix['Jungle']=='wind', 'Jungle'] = 'winds'
teams_fix.loc[teams_fix['Jungle']=='s jcain', 'Jungle'] = 'jcain'
teams_fix.loc[teams_fix['Jungle']=='tierwulf vpp', 'Jungle'] = 'tierwulf'
teams_fix.loc[teams_fix['Jungle']=='t4nk', 'Jungle'] = 't4nky'
teams_fix.loc[teams_fix['Jungle']=='rawbin iv', 'Jungle'] = 'rawbin'
teams_fix.loc[teams_fix['Jungle']=='clearlove7', 'Jungle'] = 'clearlove'
teams_fix.loc[teams_fix['Jungle']=='nintendudex', 'Jungle'] = 'nintendude'

teams_fix.loc[teams_fix['Middle']=='slooshi8', 'Middle'] = 'slooshi'
teams_fix.loc[teams_fix['Middle']=='alex ich', 'Middle'] = 'alexich'
teams_fix.loc[teams_fix['Middle']=='sorenxd', 'Middle'] = 'soren'
teams_fix.loc[teams_fix['Middle']=='niq760', 'Middle'] = 'niq'
teams_fix.loc[teams_fix['Middle']=='gear', 'Middle'] = 'gears'
teams_fix.loc[teams_fix['Middle']=='rakin vpp', 'Middle'] = 'rakin'
teams_fix.loc[teams_fix['Middle']=='longb gncm', 'Middle'] = 'longb'
teams_fix.loc[teams_fix['Middle']=='godbro sama', 'Middle'] = 'godbro'
teams_fix.loc[teams_fix['Middle']=='soz purefect', 'Middle'] = 'sozpurefect'
teams_fix.loc[teams_fix['Middle']=='takefuned', 'Middle'] = 'takefun'

teams_fix.loc[teams_fix['ADC']=='forg1vengre', 'ADC'] = 'forg1ven'
teams_fix.loc[teams_fix['ADC']=='steeelbackkk', 'ADC'] = 'steeelback'
teams_fix.loc[teams_fix['ADC']=='s0nstar', 'ADC'] = 'sonstar'
teams_fix.loc[teams_fix['ADC']=='kkramer', 'ADC'] = 'kramer'
teams_fix.loc[teams_fix['ADC']=='deee', 'ADC'] = 'dee'
teams_fix.loc[teams_fix['ADC']=='M17dee', 'ADC'] = 'dee'
teams_fix.loc[teams_fix['ADC']=='madnesss', 'ADC'] = 'madness'
teams_fix.loc[teams_fix['ADC']=='brtt rx', 'ADC'] = 'brtt'
teams_fix.loc[teams_fix['ADC']=='sacyr', 'ADC'] = 'sacy'
teams_fix.loc[teams_fix['ADC']=='the foxz', 'ADC'] = 'thefoxz'
teams_fix.loc[teams_fix['ADC']=='killeres', 'ADC'] = 'killers'
teams_fix.loc[teams_fix['ADC']=='ruvelius376', 'ADC'] = 'ruvelius'
teams_fix.loc[teams_fix['ADC']=='achuu cop', 'ADC'] = 'achuu'
teams_fix.loc[teams_fix['ADC']=='achu', 'ADC'] = 'achuu'
teams_fix.loc[teams_fix['ADC']=='yutorimoyasi', 'ADC'] = 'yutorimoyashi'
teams_fix.loc[teams_fix['ADC']=='vincentvega', 'ADC'] = 'vincent'

teams_fix.loc[teams_fix['Support']=='gleebglarbu', 'Support'] = 'gleeb'
teams_fix.loc[teams_fix['Support']=='bunny fufuu', 'Support'] = 'bunnyfufuu'
teams_fix.loc[teams_fix['Support']=='sweetr', 'Support'] = 'sweet'
teams_fix.loc[teams_fix['Support']=='s ysera', 'Support'] = 'ysera'
teams_fix.loc[teams_fix['Support']=='wei', 'Support'] = 'weiiii'
teams_fix.loc[teams_fix['Support']=='candybb', 'Support'] = 'candy'
teams_fix.loc[teams_fix['Support']=='woswos', 'Support'] = 'wos'
teams_fix.loc[teams_fix['Support']=='redbert vpp', 'Support'] = 'redbert'
teams_fix.loc[teams_fix['Support']=='rogu sama', 'Support'] = 'rogu'
teams_fix.loc[teams_fix['Support']=='regret9', 'Support'] = 'regret'
teams_fix.loc[teams_fix['Support']=='santas', 'Support'] = 'sants'
teams_fix.loc[teams_fix['Support']=='archie2b', 'Support'] = 'archie'

In [4]:
len(teams_fix['Top'].unique())

313

In [5]:
teams_no_dupes = teams_fix.drop_duplicates(subset=['TeamTag','Top','Jungle','Middle', 'ADC', 'Support'])
teams_no_dupes.shape

(1203, 10)

In [20]:
teams_no_dupes = teams_no_dupes.sort_values(['Year', 'Season', 'Type'], ascending=[1, 1, 1])

In [21]:
team_FNC = teams_no_dupes[teams_no_dupes['TeamTag']=='FNC']
team_FNC

,League,Year,Season,Type,TeamTag,Top,Jungle,Middle,ADC,Support
6420,WC,2014,Summer,International,FNC,soaz,cyanide,xpeke,rekkles,yellowstar
1122,EULCS,2015,Spring,Season,FNC,huni,reignover,febiven,steeelback,yellowstar
1213,EULCS,2015,Summer,Season,FNC,huni,reignover,febiven,rekkles,yellowstar
1357,EULCS,2016,Spring,Season,FNC,gamsu,spirit,febiven,rekkles,noxiak
1398,EULCS,2016,Spring,Season,FNC,gamsu,spirit,febiven,rekkles,klaj
1451,EULCS,2016,Summer,Season,FNC,gamsu,spirit,febiven,rekkles,yellowstar
1571,EULCS,2016,Summer,Season,FNC,kikis,spirit,febiven,rekkles,yellowstar
1616,EULCS,2016,Summer,Season,FNC,werlyb,spirit,febiven,rekkles,yellowstar
1691,EULCS,2017,Spring,Season,FNC,soaz,amazing,caps,rekkles,jesiz
1744,EULCS,2017,Spring,Season,FNC,soaz,broxah,caps,rekkles,jesiz


In [24]:
# Lo que voy a hacer es eliminar los subs, es decir, jugadores que sólo juegan un partido en total con el equipo.
teams_EULCS = teams_fix[teams_fix['League'] == 'EULCS']
teams_EULCS.groupby(['TeamTag','Top','Jungle','Middle', 'ADC', 'Support'], axis=0).count()

League  \
TeamTag Top           Jungle        Middle       ADC         Support                
ASC     zvene         mister        flezeth      ladooscuro  angelas            2   
CF      cejlon        idyiom        shiage       sayn        tarky              1   
        idyiom        shiage        sayn         tarky       cejlon             1   
CW      lenny         shook         soren        freeze      je suis kaas       8   
        wickd         k0u           caedrel      p1noy       sirnukesalot       5   
        youngbuck     airwaks       soren        freeze      unlimited         29   
                                                 woolite     unlimited          3   
                      shook         soren        freeze      unlimited          2   
CWA     morsu         kirei         cozq         vizility    hybrid             3   
DD      warriorlady   anonpsycko    magebane     dom1nant    wildpanda          4   
EL      jwaow         dexter        froggen      tabzz       nyph              10   
                                                             promisq            8   
        kev1n         shook         froggen      rekkles     krepo              2   
                                                             nyph               2   
        steve         gilius        eika         mrrallez    sprattel          18   
        wickd         shook         froggen      rekkles     krepo              6   
                                                             nyph               8   
FAC     xaxus         obvious       sozpurefect  sedrion     mountain          12   
FNA     kikis         amazing       nisqy        mrrallez    klaj              12   
FNC     gamsu         spirit        febiven      rekkles     klaj              23   
                                                             noxiak             8   
                                                             yellowstar        24   
        huni          reignover     febiven      rekkles     yellowstar        26   
                                                 steeelback  yellowstar        28   
        kikis         spirit        febiven      rekkles     yellowstar        17   
        soaz          amazing       caps         rekkles     jesiz             11   
                      broxah        caps         rekkles     hylissang          4   
                                                             jesiz             76   
        werlyb        spirit        febiven      rekkles     yellowstar         2   
G2      expect        trick         perkz        zven        mithy            119   
...                                                                           ...   
SK      fredy122      svenskeren    fox          candypanda  nrated            18   
                                                 forg1ven    nrated            28   
SKP     beansu        trashy        godbro       steeelback  barneyd            3   
        smittyj       kikis         watdefox     zyzz        zytan              4   
SPA     steve         shaunz        shlaya       nono        lounet             2   
SPY     odoamne       xerxe         nisqy        kobbe       kasing             4   
        wunder        trashy        sencux       kobbe       mikyx            122   
                                                             nisbeth           23   
TRC     babunia       mowarth       tiridus      sa1na       niko3333           3   
TS      k3nst3r       maybenickname heavenstrike siralphaq   vioreleeee         1   
        maybenickname k3nst3r       heavenstrike siralphaq   vioreleeee         1   
UOL     vizicsacsi    diamondprox   fox          steeelback  hylissang          4   
                      djoko         fox          steeelback  hylissang          2   
                      gilius        powerofevil  vardags     hylissang          9   
                      h0r0          powerofevil  vardags     hylissang         1

# En realidad, lo que no quiero es cargarme los dupes y luego añadir pesos, ya "podaremos" eliminando los pesos menores a cierta cifra.

In [28]:
teams_EULCS = teams_fix[(teams_fix['League'] == 'EULCS') & (teams_fix['Year'] != 2018)]

In [29]:
team_G2 = teams_EULCS[teams_EULCS['TeamTag']=='G2']
team_FNC = teams_EULCS[teams_EULCS['TeamTag']=='FNC']
team_G2

,League,Year,Season,Type,TeamTag,Top,Jungle,Middle,ADC,Support
1366,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1376,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1379,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1389,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1392,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1400,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1417,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1432,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1446,EULCS,2016,Spring,Season,G2,kikis,trick,perkz,emperor,hybrid
1448,EULCS,2016,Summer,Season,G2,kikis,trick,perkz,zven,mithy


In [30]:
G_G2=nx.Graph()

for index, row in team_G2.iterrows():
    
    teammates = [row['Top'], row['Jungle'], row['Middle'], row['ADC'], row['Support']]
    
    for a, b in itertools.combinations(teammates, 2):
        if G_G2.has_edge(a,b):
            G_G2[a][b]['weight'] += 1 
        else:
            G_G2.add_edge(a,b, weight = 1)
            
nx.write_graphml(G_G2, "Team_G2.graphml")

In [31]:
G_FNC=nx.Graph()

for index, row in team_FNC.iterrows():
    
    teammates = [row['Top'], row['Jungle'], row['Middle'], row['ADC'], row['Support']]
    
    for a, b in itertools.combinations(teammates, 2):
        if G_FNC.has_edge(a,b):
            G_FNC[a][b]['weight'] += 1 
        else:
            G_FNC.add_edge(a,b, weight = 1)
            
nx.write_graphml(G_FNC, "Team_FNC.graphml")

In [32]:
G_EULCS=nx.Graph()

for index, row in teams_EULCS.iterrows():
    
    teammates = [row['Top'], row['Jungle'], row['Middle'], row['ADC'], row['Support']]
    
    for a, b in itertools.combinations(teammates, 2):
        if G_EULCS.has_edge(a,b):
            G_EULCS[a][b]['weight'] += 1 
        else:
            G_EULCS.add_edge(a,b, weight = 1)
            
nx.write_graphml(G_EULCS, "Teams_EULCS.graphml")